In [1]:
spark

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 47, 6, Finished, Available)

# Set up data configuration

In [8]:
blob_account_name = "marckvnonprodblob"
blob_container_name = "bigdata"
# read only
blob_sas_token = "?sv=2021-10-04&st=2023-10-04T01%3A42%3A59Z&se=2024-01-02T02%3A42%3A00Z&sr=c&sp=rlf&sig=w3CH9MbCOpwO7DtHlrahc7AlRPxSZZb8MOgS6TaXLzI%3D"

wasbs_base_url = (
    f"wasbs://{blob_container_name}@{blob_account_name}.blob.core.windows.net/"
)
spark.conf.set(
    f"fs.azure.sas.{blob_container_name}.{blob_account_name}.blob.core.windows.net",
    blob_sas_token,
)

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 48, 10, Finished, Available)

# Reading in single parquet file

In [9]:
submissions_path = "reddit-parquet/submissions/"

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 48, 11, Finished, Available)

# select subreddit topics we are going to analyze

In [10]:
topic = ["Tetris","pokemon","SuperMario","GTA","CallOfDuty","FIFA","legostarwars", "assassinscreed","thesims","FinalFantasy"]

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 48, 12, Finished, Available)

# Reading in all of the Reddit data

In [11]:
submissions_df = spark.read.parquet(f"{wasbs_base_url}{submissions_path}")

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 48, 13, Finished, Available)

In [12]:
from pyspark.sql.functions import col, asc,desc

submissions_filtered = submissions_df.filter(col("subreddit").isin(topic))

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 48, 14, Finished, Available)

# select post with the subreddit topics

In [26]:
df_post = df_save = submissions_filtered.select("subreddit","selftext").cache()

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 48, 28, Finished, Available)

In [73]:
df_post1 = submissions_filtered.select("subreddit","title","selftext").cache()

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 40, 70, Finished, Available)

# spark NLP pipeline build

In [3]:
%%configure -f \
{"conf": {"spark.jars.packages": "com.johnsnowlabs.nlp:spark-nlp_2.12:5.1.2"}}

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 48, -1, Finished, Available)

Unrecognized options: 

In [4]:
!pip install spark-nlp

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 48, 6, Finished, Available)

     |████████████████████████████████| 540 kB 10.1 MB/s eta 0:00:01


In [14]:
import pandas as pd
import numpy as np
import json
from pyspark.ml import Pipeline
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from sparknlp.annotator import *
from sparknlp.base import *
import sparknlp
from sparknlp.pretrained import PretrainedPipeline

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 48, 16, Finished, Available)

In [43]:
documentAssembler = DocumentAssembler()\
    .setInputCol("selftext")\
    .setOutputCol("document")

sentenceDetector = SentenceDetector() \
    .setInputCols(["document"]) \
    .setOutputCol("sentence")

tokenizer = Tokenizer().setInputCols(["sentence"]).setOutputCol("token")

lemmatizer = Lemmatizer().setInputCols(["token"]).setOutputCol("lem_token").setDictionary("src/test/resources/lemma-corpus-small/lemmas_small.txt", "->", "\t")

#stop_words = StopWordsCleaner().setInputCols(["lem_token"]).setOutputCol("clean_token").setCaseSensitive(False)

#stemmer = Stemmer().setInputCols(["clean_token"]).setOutputCol("stem_token")

#spell_check = NorvigSweetingModel().setInputCols(["stem_token"]).setOutputCol("final_token")


cleanPipeline = Pipeline(
      stages = [
          documentAssembler,
          sentenceDetector,
          tokenizer,
          lemmatizer
      ])


StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 48, 45, Finished, Available)

In [31]:
documentAssembler = DocumentAssembler()\
    .setInputCol("selftext")\
    .setOutputCol("document")

    
use = UniversalSentenceEncoder.pretrained(name="tfhub_use", lang="en")\
 .setInputCols(["document"])\
 .setOutputCol("sentence_embeddings")


sentimentdl = SentimentDLModel.pretrained(lang="en")\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("sentiment")

nlpPipeline = Pipeline(
      stages = [
          documentAssembler,
          use,
          sentimentdl
      ])

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 48, 33, Finished, Available)

tfhub_use download started this may take some time.
Approximate size to download 923.7 MB
[OK!]
sentimentdl_use_imdb download started this may take some time.
Approximate size to download 12 MB
[OK!]


# run the pipeline

In [27]:
df_post_filter = df_post.filter(~col("selftext").isin([" ","[removed]","[deleted]"]))

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 48, 29, Finished, Available)

In [28]:
df_post_filter = df_post_filter.dropna()

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 48, 30, Finished, Available)

In [75]:
df_post2 = df_post1.dropna()

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 40, 72, Finished, Available)

In [44]:

data = spark.createDataFrame([["Peter Pipers employees are picking pecks of pickled peppers."]]) \
    .toDF("selftext")
result = cleanPipeline.fit(data).transform(data)
result.selectExpr("lemma.result").show(truncate=False)

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 48, 46, Submitted, Running)

In [38]:
cleaned_df = cleanPipeline.fit(df_post_filter)

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 48, 40, Finished, Available)

IllegalArgumentException: form column required. Verify that training dataset was loaded with CoNLLU component

In [29]:
senti_model = nlpPipeline.fit(df_post_filter)

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 48, 31, Finished, Available)

IllegalArgumentException: form column required. Verify that training dataset was loaded with CoNLLU component

In [76]:
senti_model2 = nlpPipeline.fit(df_post2)

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 40, 73, Finished, Available)

In [30]:
senti_pred = senti_model.transform(df_post)

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 40, 27, Finished, Available)

In [77]:
senti_pred2 = senti_model.transform(df_post2)

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 40, 74, Finished, Available)

In [63]:
senti_pred.show(5)

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 40, 60, Finished, Available)

+------------+--------------------+--------------------+--------------------+--------------------+
|   subreddit|            selftext|            document| sentence_embeddings|           sentiment|
+------------+--------------------+--------------------+--------------------+--------------------+
|        FIFA|                    |[{document, 0, -1...|                  []|                  []|
|FinalFantasy|                    |[{document, 0, -1...|                  []|                  []|
|     pokemon|So several days a...|[{document, 0, 13...|[{sentence_embedd...|[{category, 0, 13...|
|        FIFA|           [removed]|[{document, 0, 8,...|[{sentence_embedd...|[{category, 0, 8,...|
|        FIFA|                    |[{document, 0, -1...|[{sentence_embedd...|[{category, 0, 8,...|
+------------+--------------------+--------------------+--------------------+--------------------+
only showing top 5 rows



In [78]:
senti_pred2.show(5)

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 40, 75, Finished, Available)

+------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|   subreddit|               title|            selftext|            document| sentence_embeddings|           sentiment|
+------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|        FIFA|if you every get ...|                    |[{document, 0, -1...|                  []|                  []|
|FinalFantasy|Final Fantasy Pix...|                    |[{document, 0, -1...|                  []|                  []|
|     pokemon|the PokemonTogeth...|So several days a...|[{document, 0, 13...|[{sentence_embedd...|[{category, 0, 13...|
|        FIFA|should i select o...|           [removed]|[{document, 0, 8,...|[{sentence_embedd...|[{category, 0, 8,...|
|        FIFA| Team of the week 17|                    |[{document, 0, -1...|[{sentence_embedd...|[{category, 0, 8,...|
+------------+--------------------+-----

In [ ]:
from pyspark.sql.functions import col, size

In [79]:
df_filtered = senti_pred2.filter(size(col("sentence_embeddings")) != 0)
df_filtered.show(5)

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 40, 76, Finished, Available)

+---------+--------------------+--------------------+--------------------+--------------------+--------------------+
|subreddit|               title|            selftext|            document| sentence_embeddings|           sentiment|
+---------+--------------------+--------------------+--------------------+--------------------+--------------------+
|  pokemon|the PokemonTogeth...|So several days a...|[{document, 0, 13...|[{sentence_embedd...|[{category, 0, 13...|
|     FIFA|should i select o...|           [removed]|[{document, 0, 8,...|[{sentence_embedd...|[{category, 0, 8,...|
|     FIFA| Team of the week 17|                    |[{document, 0, -1...|[{sentence_embedd...|[{category, 0, 8,...|
|  pokemon|Who's a non-villa...|For me, Tyme insp...|[{document, 0, 66...|[{sentence_embedd...|[{category, 0, 66...|
|  pokemon|Zard Hunting in B...|           [removed]|[{document, 0, 8,...|[{sentence_embedd...|[{category, 0, 8,...|
+---------+--------------------+--------------------+-----------

In [35]:
df_senti = senti_pred.select("subreddit",F.explode('sentiment.result'))

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 40, 32, Finished, Available)

In [82]:
df_senti2 = senti_pred2.select("subreddit",F.explode('sentiment.result'))

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 40, 79, Finished, Available)

In [57]:
df_senti = df_senti.withColumnRenamed("col", "sentiment")

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 40, 54, Finished, Available)

In [83]:
df_senti2 = df_senti2.withColumnRenamed("col", "sentiment")

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 40, 80, Finished, Available)

In [80]:
df_senti.show(5)

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 40, 77, Finished, Available)

+---------+---------+
|subreddit|sentiment|
+---------+---------+
|  pokemon|      pos|
|     FIFA|      pos|
|     FIFA|      neg|
|  pokemon|      neg|
|  pokemon|      pos|
+---------+---------+
only showing top 5 rows



In [84]:
df_senti2.show(5)

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 40, 81, Finished, Available)

+---------+---------+
|subreddit|sentiment|
+---------+---------+
|  pokemon|      pos|
|     FIFA|      pos|
|     FIFA|      neg|
|  pokemon|      neg|
|  pokemon|      pos|
+---------+---------+
only showing top 5 rows



In [37]:
df_senti_count = df_senti.groupBy("subreddit","sentiment").count().toPandas()

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 13, 30, Finished, Available)

In [85]:
df_filtered_use=df_filtered.limit(10000)
df_senti_use=df_senti2.limit(10000)

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 40, 82, Finished, Available)

In [87]:
df_filtered_use_final = df_filtered_use.drop(*("document", "sentence_embeddings", "sentiment"))
df_filtered_use_final.show(5)

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 40, 84, Finished, Available)

+---------+--------------------+--------------------+
|subreddit|               title|            selftext|
+---------+--------------------+--------------------+
|  pokemon|the PokemonTogeth...|So several days a...|
|     FIFA|should i select o...|           [removed]|
|     FIFA| Team of the week 17|                    |
|  pokemon|Who's a non-villa...|For me, Tyme insp...|
|  pokemon|Zard Hunting in B...|           [removed]|
+---------+--------------------+--------------------+
only showing top 5 rows



In [92]:
from pyspark.sql.functions import monotonically_increasing_id

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 40, 89, Finished, Available)

In [93]:
df_senti_text_use = df_filtered_use_final.withColumn("index", monotonically_increasing_id()).join(df_senti_use.withColumn("index", monotonically_increasing_id()),"index").drop("index")
df_senti_text_use.show(5)

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 40, 90, Finished, Available)

+---------+--------------------+--------------------+---------+---------+
|subreddit|               title|            selftext|subreddit|sentiment|
+---------+--------------------+--------------------+---------+---------+
|  pokemon|the PokemonTogeth...|So several days a...|  pokemon|      pos|
|     FIFA|should i select o...|           [removed]|     FIFA|      pos|
|     FIFA| Team of the week 17|                    |     FIFA|      neg|
|  pokemon|Who's a non-villa...|For me, Tyme insp...|  pokemon|      neg|
|  pokemon|Zard Hunting in B...|           [removed]|  pokemon|      pos|
+---------+--------------------+--------------------+---------+---------+
only showing top 5 rows



In [111]:
df_senti_text_save=df_senti_text_use.toPandas()

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 40, 108, Finished, Available)

In [41]:
import os
CSV_DIR = os.path.join("Users/ky285/fall-2023-reddit-project-team-11/data", "csv")
df_senti_count.to_csv(f"{CSV_DIR}/sentiment_count_top10.csv", index=False)

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 13, 34, Finished, Available)

In [112]:
import os
CSV_DIR = os.path.join("Users/zx111/fall-2023-reddit-project-team-11/data", "csv")
df_senti_text_save.to_csv(f"{CSV_DIR}/sentiment_text_top10.csv", index=False)

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 40, 109, Finished, Available)